In [1]:
import pandas as pd

In [2]:
news_data = pd.read_csv("Data/news_data.csv")

In [3]:
news_data.head()

,category,datetime,headline,id,image,related,source,summary,url
0,company,1697909459,Humanoid robots face a major test with Amazon'...,123320327,https://techcrunch.com/wp-content/uploads/2023...,AMZN,Yahoo,Announced amid a deluge of news at this week’s...,https://finnhub.io/api/news?id=8cb32870bb15be8...
1,company,1697896800,"3 Stocks That Turned $1,000 Into $1.1 Million ...",123315781,NaN,MNST,Yahoo,Consumer products have proven themselves to be...,https://finnhub.io/api/news?id=d75d610543b91f1...
2,company,1697896800,The U.S. Cities With the Most Cutting-Edge Tec...,123315249,https://s.yimg.com/ny/api/res/1.2/kLEZByvMTM7D...,WMT,Yahoo,The Seattle area has the highest proportion of...,https://finnhub.io/api/news?id=faca644c2f7e4e6...
3,company,1697895295,Dow Jones Futures: Market Point Break? Microso...,123315250,https://media.zenfs.com/en/ibd.com/ec8365e261a...,MSFT,Yahoo,"Microsoft, Meta and Google lead a massive wave...",https://finnhub.io/api/news?id=6c9b9cc8fb7f9ec...
4,company,1697890800,Rivian (NASDAQ:RIVN): Wall Street Loves This E...,123319798,NaN,TSLA,TipRanks,Looking for stock market analysis and research...,https://finnhub.io/api/news?id=fe0b89552f14e89...


In [4]:
news_data.drop(columns=['category','image','url','id','source'], inplace=True)

In [5]:
news_data.columns

Index(['datetime', 'headline', 'related', 'summary'], dtype='object')

In [6]:
news_data.shape

(757, 4)

In [7]:
news_data.head()

,datetime,headline,related,summary
0,1697909459,Humanoid robots face a major test with Amazon'...,AMZN,Announced amid a deluge of news at this week’s...
1,1697896800,"3 Stocks That Turned $1,000 Into $1.1 Million ...",MNST,Consumer products have proven themselves to be...
2,1697896800,The U.S. Cities With the Most Cutting-Edge Tec...,WMT,The Seattle area has the highest proportion of...
3,1697895295,Dow Jones Futures: Market Point Break? Microso...,MSFT,"Microsoft, Meta and Google lead a massive wave..."
4,1697890800,Rivian (NASDAQ:RIVN): Wall Street Loves This E...,TSLA,Looking for stock market analysis and research...


In [8]:
print(news_data['datetime'].dtype)

int64


In [9]:
news_data['y'] = 0

In [10]:
from fetch_stock_data_for_date import StockData

In [11]:
sd = StockData()

In [12]:
errors = []

In [13]:
def get_stock_data(x):
    try:
        return sd.get_delta(x['datetime'], x['related'])
    except ValueError as ve:
        print(ve)
        errors.append(x)
        return None

In [28]:
news_data['y'] = news_data.apply(lambda x : get_stock_data(x),axis=1)

In [29]:
news_data['y'].describe()

count    700.000000
mean       0.828820
std       19.750677
min     -136.899902
25%       -2.850010
50%       -0.550003
75%        0.579998
max      440.360107
Name: y, dtype: float64

In [30]:
news_data[~news_data['y'].notna()]

,datetime,headline,related,summary,y
538,1699742439,IWB: Mega-Cap Tech Powers US Large Caps Higher...,AAPL,Discover why mega-cap tech stocks are boosting...,NaN
539,1699741522,CII And ETV: 2 CEFs Worth Buying When They're ...,AAPL,CII And ETV have put up solid results and prov...,NaN
554,1699676040,The 3 Priciest Magnificent Seven Stocks: Worth...,AAPL,Looking for stock market analysis and research...,NaN
555,1699665600,The Stagflation Storm: A Comprehensive Look At...,AAPL,Looking for stock market analysis and research...,NaN
556,1699664460,Optimize Your Portfolio Review,AAPL,Looking for stock market analysis and research...,NaN
557,1699663940,Hollywood actors to vote on strike deal after ...,AAPL,"The actors union, SAG-AFTRA, revealed the deta...",NaN
558,1699660680,Weekly Roundup,AAPL,We made no additions to the portfolio nor did ...,NaN
559,1699650000,Buy These 2 Growth Stocks on the Dip,AAPL,Apple and Disney have faced repeated challenge...,NaN
560,1699647509,Apple's ad-free Amazon presence raises questions,AAPL,Apple's Amazon product pages have a minimalist...,NaN
561,1699647440,Can You Buy Penny Stocks In An IRA? The Answer...,AAPL,You can buy penny stocks with an IRA and here'...,NaN
